<a href="https://colab.research.google.com/github/makhmudovamunira/DataScience_Mohirdev/blob/main/IerarxikCluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%204/data/cars_clus.csv"
df = pd.read_csv(url)
df.head()

Datasetni tozalaymiz

In [ ]:
df[[ 'sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']] = df[['sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']].apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df = df.reset_index(drop=True)
df.head(5)

Klaster uchun ustunlarni tanlab olamiz

In [ ]:
featureset = df[['engine_s',  'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap', 'mpg']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler=MinMaxScaler()
x=featureset.values
feature_mtx=min_max_scaler.fit_transform(x)
feature_mtx[0:5]

## Klasterlash

Ierarxik klasterlash uchun `Scipy` paketidan foydalanmiz.

Boshlanishiga distance matrix `D` (masofalar matrisasini) quramiz:

In [ ]:
import scipy
from scipy import ndimage
from scipy.spatial import distance_matrix
from scipy.cluster import hierarchy

leng=feature_mtx.shape[0]
D=np.zeros([leng,leng])
for i in range(leng):
  for j in range(leng):
    D[i,j]=scipy.spatial.distance.euclidean(feature_mtx[i], feature_mtx[j])

D


Modelni o'qitamiz (training)

In [ ]:
Z=hierarchy.linkage(D)

In [ ]:
fig=plt.figure(figsize=(18,50))
def llf(id):
  return '[%s %s %s]' % (df['manufact'][id], df['model'][id], int(float(df['type'][id])))
dendro=hierarchy.dendrogram(Z, leaf_label_func=llf, leaf_font_size=12, orientation='right')

Klasterlar sonini o'zimiz belgilab, dendogramni qayerdan kesishni ko'rsatishimiz mumkin.

In [ ]:
from scipy.cluster.hierarchy import fcluster
k = 5 # 5 ta klaster
clusters = fcluster(Z, k, criterion='maxclust')
clusters

In [ ]:
from sklearn import manifold, datasets
from sklearn.cluster import AgglomerativeClustering

## `scikit` yordamida klasterlash

In [ ]:
agglom=AgglomerativeClustering(n_clusters=6, linkage='complete')
agglom.fit(D)
agglom.labels_

In [ ]:
df['cluster']=agglom.labels_
agg_cars=df.groupby(['cluster','type'])[['horsepow','engine_s','mpg','price']].mean()
agg_cars